<a href="https://colab.research.google.com/github/blindTissue/nlp_hw7/blob/main/bloomz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("boolq")

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
train_dataset = dataset["train"]
true_instances = train_dataset.filter(lambda x: x['answer'] == True)
false_instances = train_dataset.filter(lambda x: x['answer'] == False)

import random


def get_balanced_string(length=4):
    out = ""
    for i in range(length // 2):
        true_item = true_instances[random.randrange(len(true_instances))]
        false_item = false_instances[random.randrange(len(false_instances))]
        out += 'Q: ' + true_item['question'] + '? P: ' + true_item['passage'] + ". A: " + str(true_item['answer']) + '. '
        out += 'Q: ' + false_item['question'] + '? P: ' + false_item['passage'] + ". A: " + str(false_item['answer'])+ '. '
    return out


def get_random_query(t):
    if t:
        item = true_instances[random.randrange(len(true_instances))]
    else:
        item = false_instances[random.randrange(len(false_instances))]
    return 'Q: ' + item['question'] + '? P: ' + item['passage']


In [4]:
!pip install -q transformers accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto")
#model.to("cuda")
inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=400)
print(tokenizer.decode(outputs[0]))


Translate to English: Je t’aime. I love you.</s>


In [5]:
shots = get_balanced_string(8)
q_t = get_random_query(True)
inputs = tokenizer.encode(shots + q_t, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=400)
print(tokenizer.decode(outputs[0][-10:]))
print(q_t)

. Andrea eventually reanimates as walker and tries
Q: can the xbox one x play 360 games? P: The functionality will be similar to that for back-compatibility with Xbox 360 games. Users insert the Xbox game disc into their Xbox One console to install the compatible version of the game. While players will not be able to access any old game saves or connect to Xbox Live on these titles, system link functions will remain available. Xbox games will not receive achievement support, although when asked about this component, Spencer responded that they had nothing to announce at the current time.
